In [ ]:
# router is a file with api keys configured
from router import router_api
router_api('mining')

In [4]:
import logging

from langchain_openai import ChatOpenAI
from sisyphus.chain import Paragraph
from sisyphus.heas.extract_lc import extract
from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.chain.chain_elements import Filter, Writer, run_chains_with_extarction_history_multi_threads

logger = logging.getLogger(__name__)
logger.setLevel(10)
fh = logging.FileHandler('heas.log')
fh.setLevel(10)
ft = logging.Formatter('%(message)s')
fh.setFormatter(ft)
logger.addHandler(fh)

model = ChatOpenAI(temperature=0, model='gpt-4.1', max_tokens=15000)

In [5]:
database_name = 'labeled_'
result_db_name = 'heas_context_test'
target_dir = 'TEST_FILES'

In [6]:
db = get_plain_articledb(database_name)
getter = Filter(db)
result_db = get_create_resultdb(result_db_name)
writer = Writer(result_db)
 
def load(docs):
    return [Paragraph.from_labeled_document(doc, id_) for id_, doc in enumerate(docs)]

from functools import partial
extract_p = partial(extract, extraction_model=model)
chain = getter + load + extract_p + writer
debug_chain = getter + load + extract_p

/Users/pastalover/miniconda3/envs/sisyphus_context/lib/python3.10/site-packages/pydantic/main.py:1552: RuntimeWarning: fields may not start with an underscore, ignoring "__tablename__"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


In [7]:
debug_chain.compose('10.1016&sol;j.msea.2023.144779.html')

'failed'

In [5]:
run_chains_with_extarction_history_multi_threads(chain, target_dir, 5, result_db_name)

100%|██████████| 100/100 [04:14<00:00,  2.54s/it]
